In [ ]:
import pandas as pd

data_source = pd.read_csv('RF.csv')

,DailyTotalRainfall_AllYear_url,DailyTotalRainfall_CurrentYear_url,WeatherStationName_en,WeatherStationName_sc,WeatherStationName_uc,GeometryLongitude,GeometryLatitude
0,https://data.weather.gov.hk/weatherAPI/hko_dat...,https://data.weather.gov.hk/weatherAPI/hko_dat...,Hong Kong International Airport,香港国际机场,香港國際機場,113.921944,22.309444
1,https://data.weather.gov.hk/weatherAPI/hko_dat...,https://data.weather.gov.hk/weatherAPI/hko_dat...,Cheung Chau,长洲,長洲,114.026667,22.201111
2,https://data.weather.gov.hk/weatherAPI/hko_dat...,https://data.weather.gov.hk/weatherAPI/hko_dat...,Hong Kong Observatory,香港天文台,香港天文台,114.174167,22.301944
3,https://data.weather.gov.hk/weatherAPI/hko_dat...,https://data.weather.gov.hk/weatherAPI/hko_dat...,Kai Tak,启德,啟德,114.213333,22.309722
4,https://data.weather.gov.hk/weatherAPI/hko_dat...,https://data.weather.gov.hk/weatherAPI/hko_dat...,Kau Sai Chau,滘西洲,滘西洲,114.312500,22.370278
5,https://data.weather.gov.hk/weatherAPI/hko_dat...,https://data.weather.gov.hk/weatherAPI/hko_dat...,King's Park,京士柏,京士柏,114.172778,22.311944
6,https://data.weather.gov.hk/weatherAPI/hko_dat...,https://data.weather.gov.hk/weatherAPI/hko_dat...,Lau Fau Shan,流浮山,流浮山,113.983611,22.468889
7,https://data.weather.gov.hk/weatherAPI/hko_dat...,https://data.weather.gov.hk/weatherAPI/hko_dat...,Tsak Yue Wu,鲫鱼湖,鯽魚湖,114.323056,22.402778
8,https://data.weather.gov.hk/weatherAPI/hko_dat...,https://data.weather.gov.hk/weatherAPI/hko_dat...,Peng Chau,坪洲,坪洲,114.043333,22.291111
9,https://data.weather.gov.hk/weatherAPI/hko_dat...,https://data.weather.gov.hk/weatherAPI/hko_dat...,Sha Tin,沙田,沙田,114.210000,22.402500
